# Retriever-Reader System

Imports and Constants

In [1]:
# Imports
import torch
import re
import os
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from datasets import load_dataset

# Constans
cache_dir = '/usr/src/app/!datasets/'
os.environ["TRANSFORMERS_CACHE"] = cache_dir
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

Model declaratives:

In [2]:
class RRSystem:
    def __init__(self, retriever='DeepPavlov/rubert-base-cased', reader='AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru', cache_dir='~/.cache/huggingface/datasets'):
        # Create pretrained models and sending to device
        self.tokenizer_retriever = BertTokenizer.from_pretrained(retriever, do_lower_case=True, cache_dir=cache_dir)
        self.model_retriever = BertModel.from_pretrained(retriever, output_hidden_states = True, cache_dir=cache_dir)
        self.model_retriever.to(device)
        self.model_retriever.eval()

        self.tokenizer_reader = AutoTokenizer.from_pretrained(reader, cache_dir=cache_dir)
        self.model_reader = AutoModelForQuestionAnswering.from_pretrained(reader, cache_dir=cache_dir)
        self.model_reader.to(device)
        self.model_reader.eval()

    def load_text(self, texts):
        self.texts = texts
        # Text2Vec translate
        self.texts_vectors = [self._text2vec(text).numpy() for text in texts]
    
    def _text2vec(self, text):
        inputs = self.tokenizer_retriever(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {name: tensor.to(device) for name, tensor in inputs.items()}
        outputs = self.model_retriever(**inputs)
        return outputs.last_hidden_state.mean(dim=1).detach().cpu()

    def retriever(self, question):
        question_vector = self._text2vec(question).numpy()
        # Cosaine criterion of texts
        similarities = [cosine_similarity(text_vector, question_vector) for text_vector in self.texts_vectors]
        
        max_similarity_index = similarities.index(max(similarities))
        return self.texts[max_similarity_index], max_similarity_index

    def reader(self, question, text):
        encoding = self.tokenizer_reader.encode_plus(text=question,text_pair=text)
        inputs = encoding['input_ids']
        tokens = self.tokenizer_reader.convert_ids_to_tokens(inputs)
        inputs = torch.tensor([inputs]).to(device)
        output = self.model_reader(input_ids=inputs)

        start_scores = output[0][0]
        end_scores = output[1][0]
        max_score = float('-inf')
        start_index = end_index = 0

        for i in range(len(start_scores)):
            for j in range(i, len(end_scores)):
                score = start_scores[i] + end_scores[j]
                if score > max_score:
                    max_score = score
                    start_index = i
                    end_index = j
        answer_tokens = tokens[start_index:end_index+1]
        answer = self.tokenizer_reader.convert_tokens_to_string(answer_tokens)
        return answer

    def _preprocess_text(self, text):
        text = re.sub(r'[^\w\s]$', '', text)
        text = text.strip() + '.'
        text = text[0].upper() + text[1:]
        return text

    def __call__(self, question):
        retrieved_text, text_index = self.retriever(question)
        answer = self.reader(retrieved_text, question)
        answer = self._preprocess_text(answer)
        return answer, text_index

Dataset selection

In [3]:
dataset = load_dataset("sberquad")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [5]:
# Texts of test
def get_unique_texts(texts):
    return list(set(texts))

texts = [item['context'] for item in dataset['test']]
texts = get_unique_texts(texts)
len(texts)

4425

Model initialization and Dataset preprocessing

In [ ]:
qa_system = RRSystem(cache_dir=cache_dir)
qa_system.load_text(texts)

By my test

In [20]:
for i in range(0, 20):
    print(f"{i}) {texts[i]}")

0) Дозировка минеральных вод при питьевом лечении зависит от их химического состава, минерализации, а также от вида заболевания и состояния больного. При содержании 2-10 г солей в литре (обычные воды малой и средней минерализации) приём минеральной воды назначают три раза в день до еды по 200—250 миллилитров (1—1,5 стакана), но когда организм больного ослаблен, начинают с меньшей дозы — 50—100 мл (0,5 стакана), с последующим увеличением её до обычной. Такая методика применяется также при склонности к поносам и неустойчивой сердечно-сосудистой деятельности.
1) Мировые деньги. Внешнеторговые связи, международные займы, оказание услуг внешнему партнёру вызвали появление мировых денег. Они функционируют как всеобщее платёжное средство, всеобщее покупательное средство и всеобщая материализация общественного богатства. Мировыми деньгами обычно считают резервные валюты (в настоящее время это доллар США, швейцарский франк, евро, английский фунт, японская иена). Но для прямых международных плат

In [23]:
questions = [
    "Что такое гармонические колебания?",
    "Из чего состоит литий-ионный аккумулятор?",
    "Какие планеты обладают атмосферой?",
    "Какие породы собак официально признаны опасными для человека?",
    "Что такое однодольные растения?",
    "Что известно про 2014 год?"
            ]

idxs = []
for i, question in enumerate(questions):
    answer, idx = qa_system(question)
    idxs.append(idx)
    print(f"{i}) {question}\nОтвет: {answer} [текст №{idx}]")

0) Что такое гармонические колебания?
Ответ: Орбитальные элементы. [текст №2808]
1) Из чего состоит литий-ионный аккумулятор?
Ответ: Из электродов. [текст №1576]
2) Какие планеты обладают атмосферой?
Ответ: Юпитер и Сатурн, Нептун и Плутон) находятся в орбитальном резонансе друг с другом или с более мелкими телами (что также характерно для спутниковых систем). Все планеты, за исключением Венеры и Меркурия. [текст №1344]
3) Какие породы собак официально признаны опасными для человека?
Ответ: Питбультерьеров, тоса-ину или японских мастифов, аргентинских догов и фила бразильеро. [текст №1989]
4) Что такое однодольные растения?
Ответ: Современные семейства Кувшинковые или Перечные. [текст №2703]
5) Что известно про 2014 год?
Ответ: Неоднократно показывались спортивные трансляции. [текст №1108]


In [26]:
for i in idxs:
    print(f"[текст №{i}] {texts[i]}")

[текст №2808] Определение орбит небесных тел является одной из задач небесной механики. Для задания орбиты спутника планеты, астероида или Земли используют так называемые орбитальные элементы . Орбитальные элементы отвечают за задание базовой системы координат (точки отсчёта, о́си координат), формы и размера орбиты, её ориентации в пространстве и момент времени, в который небесное тело находится в определённой точке орбиты. В основном используются два способа задания орбиты (при наличии системы координат)[1]:
[текст №1576] Литий-ионный аккумулятор состоит из электродов (катодного материала на алюминиевой фольге и анодного материала на медной фольге), разделенных пропитанными электролитом пористыми сепараторами. Переносчиком заряда в литий-ионном аккумуляторе является положительно заряженный ион лития, который имеет способность внедряться (интеркалироваться) в кристаллическую решетку других материалов (например, в графит, окислы и соли металлов) с образованием химической связи, например